<a href="https://colab.research.google.com/github/AnushkaMazumdar2/Natural-Language-Processing/blob/main/Anushka_505_Lab4(LLM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Libraries**

In [2]:
%pip install transformers torch datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

**Importing Necessary Libraries**

In [3]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline

**Importing Dataset**

We need a labeled dataset for sentiment analysis. The IMDb movie reviews dataset is a popular choice.IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. A set of 25,000 highly polar movie reviews for training and 25,000 for testing is provided.

Label_0 means negative sentiment

Label_1 means positive sentiment



In [4]:
dataset = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

**Model and Tokenizer Initialization**


Load the pre-trained BERT model and tokenizer.

In [5]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Data Preprocessing**

Tokenize the dataset and prepare it for the model.

In [6]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

**Data Collation**

Create a data collator for batching the tokenized inputs.

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Training and Validation Split**

Split the dataset into training and validation sets.

In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select([i for i in list(range(5000))])
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select([i for i in list(range(1000))])

**Training Arguments**

Set up the training arguments.

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**Trainer Initialization**

Create a Trainer instance for the training process.


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

**Model Training**

Train the model using the Trainer instance.

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,0.249447
2,0.293700,0.293797
3,0.293700,0.355087


TrainOutput(global_step=939, training_loss=0.20304040634594026, metrics={'train_runtime': 1550.7458, 'train_samples_per_second': 9.673, 'train_steps_per_second': 0.606, 'total_flos': 3946665830400000.0, 'train_loss': 0.20304040634594026, 'epoch': 3.0})

**Evaluation**

Evaluate the fine-tuned model on the validation set.

In [12]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3550865054130554, 'eval_runtime': 32.1208, 'eval_samples_per_second': 31.132, 'eval_steps_per_second': 1.961, 'epoch': 3.0}


**Saving the Model**

Save the fine-tuned model and tokenizer for future use.

In [ ]:
model.save_pretrained("./sentiment-model")
tokenizer.save_pretrained("./sentiment-model")

**Testing**

Testing the Model using Text

In [14]:
sentiment_model = pipeline("sentiment-analysis", model="./sentiment-model", tokenizer="./sentiment-model")
result = sentiment_model("I love this movie!")
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.995756208896637}]


The model is correctly able to predict that the sentiment is positive.

In [15]:
result = sentiment_model("I hate this movie!")
print(result)

[{'label': 'LABEL_0', 'score': 0.9950624108314514}]


The model is correctly able to predict that the sentiment is negative.

In [16]:
result = sentiment_model("The movie was alright!")
print(result)

[{'label': 'LABEL_1', 'score': 0.9776856303215027}]
